# 🛠 Deploy Best AutoML Model & Test Endpoint
This notebook deploys the best candidate from a completed SageMaker Autopilot job and runs test predictions.

## 🔐 Step 1: Set AWS Credentials

In [ ]:

aws_access_key = 'YOUR_AWS_ACCESS_KEY_ID'
aws_secret_key = 'YOUR_AWS_SECRET_ACCESS_KEY'
region_name = 'us-east-1'


In [ ]:
!pip install boto3 sagemaker pandas --quiet

In [ ]:

import boto3
import sagemaker
from sagemaker.automl.automl import AutoML
from sagemaker.session import Session
from sagemaker import get_execution_role
import os
import pandas as pd

# Set environment credentials
os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key

session = sagemaker.Session()
role = get_execution_role()


## 🚀 Step 2: Attach to Completed AutoML Job

In [ ]:

# Replace with your actual AutoML job name
auto_ml_job_name = 'churn-automl-YYYY-MM-DD-HH-MM-SS'  # <-- Replace this!

automl = AutoML.attach(auto_ml_job_name=auto_ml_job_name)
best_candidate = automl.describe_auto_ml_job()['BestCandidate']
print("✅ Best candidate selected.")


## 📦 Step 3: Deploy the Model

In [ ]:

predictor = automl.deploy(initial_instance_count=1,
                          instance_type="ml.m5.large",
                          candidate=best_candidate)

print(f"✅ Model deployed to endpoint: {predictor.endpoint_name}")


## 🧪 Step 4: Test with Sample Data

In [ ]:

# Load same dataset and drop target column
df = pd.read_csv("churn.csv")
X_sample = df.drop(columns=["Churn"]).iloc[:5]  # use first 5 rows

predictions = predictor.predict(X_sample.to_dict(orient="records"))
print("📈 Predictions:", predictions)


## ✅ Cleanup (Optional)

In [ ]:

# Delete the endpoint when done
# predictor.delete_endpoint()
